In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
import timeit
import matplotlib.pyplot as plt
import pickle
from mnist import MNIST
from time import time
import numpy as np

In [3]:
print("Loading dataset....")
mndata = MNIST('../mnist_dataset')
X_train, y_train = mndata.load_training()
X_train = (mndata.process_images_to_numpy(X_train)/255)
X_test, y_test = mndata.load_testing()
X_test = (mndata.process_images_to_numpy(X_test)/255)
print("Done.")

Loading dataset....
Done.


In [4]:
def one_hot_encode(labels):
    y_inp=np.zeros((len(labels),10))
    for ind,val in enumerate(labels):
        y_inp[ind][val]=1
    return y_inp

In [5]:
def batch_gen(st=0,size=20,validate=False):
    st=st%60000
    if validate:
        X=X_test[st:st+size].reshape(size,784)
        labels=y_test[st:st+size]
    else:
        X=X_train[st:st+size].reshape(size,784)
        labels=y_train[st:st+size]
    y=one_hot_encode(labels)
    return X,y

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [15]:
model = Sequential()
model.add(Dense(128, input_dim=784, activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_4 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                330       
Total params: 104,938
Trainable params: 104,938
Non-trainable params: 0
_________________________________________________________________


In [18]:
from tensorflow.keras.optimizers import SGD
opt=SGD(learning_rate=0.1)

In [19]:
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])

In [20]:
model.fit(X_train, one_hot_encode(y_train), epochs=5, batch_size=16)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 7s 124us/sample - loss: 0.0914 - accuracy: 0.1712
Epoch 2/5
60000/60000 [==============================] - 7s 124us/sample - loss: 0.0879 - accuracy: 0.3495
Epoch 3/5
60000/60000 [==============================] - 7s 122us/sample - loss: 0.0837 - accuracy: 0.4520
Epoch 4/5
60000/60000 [==============================] - 8s 126us/sample - loss: 0.0732 - accuracy: 0.5632
Epoch 5/5
60000/60000 [==============================] - 7s 122us/sample - loss: 0.0606 - accuracy: 0.6481


In [12]:
predictions = model.predict_classes(X_train)